In [1]:
# import section
import pyomo.environ as pyo
import numpy as np
import pandas as pd

In [2]:
student_size = 30
prof_size = 12

In [21]:
model = pyo.ConcreteModel()

# sets
profs = list(range(prof_size))
students = list(range(student_size))
model.add_component('profs', pyo.Set(initialize=profs))
model.add_component('students', pyo.Set(initialize=students))

# parameters
# TODO
capacities = pd.read_csv('./data/cap.csv', index_col=0).to_numpy()
def capacities_rule(model, i):
    return capacities[i,0]
model.add_component('cap', pyo.Param(model.profs, initialize=capacities_rule))

prof_preferences = pd.read_csv('./data/prof_pref.csv', index_col=0).to_numpy()
def prof_preferences_rule(model, i, j):
    return prof_preferences[i,j]
model.add_component('prof_pref', pyo.Param(model.profs, model.students, initialize=prof_preferences_rule))

std_preferences = pd.read_csv('./data/std_pref.csv', index_col=0).to_numpy()
def std_preferences_rule(model, i, j):
    return std_preferences[i,j]
model.add_component('std_pref', pyo.Param(model.students, model.profs, initialize=std_preferences_rule))


#variables
# TODO
model.add_component('delta', pyo.Var(model.profs, model.students, domain=pyo.Binary))

# objective function
# TODO

# constraints
def single_prof_const_rule(model, j):
    return sum(model.delta[i,j] for i in model.profs) == 1
model.add_component('single_prof_per_student_const', pyo.Constraint(model.students, rule=single_prof_const_rule))

def prof_capacity_const_rule(model, i):
    return sum(model.delta[i,j] for j in model.students) == model.cap[i]
model.add_component('prof_capacity_const', pyo.Constraint(model.profs, rule=prof_capacity_const_rule))
